# Optum Data Science Challenge

These notebooks contains my method for Data Science Skills Evaluation Project. I show my solution but aim to also show my thought process and how I would typically deal with 'real' data problems like this. There is far more things that could have be done such as: additional data quality checks, more features engineering, different ML algorithms and deployment options. My aim was to show the breadth of my knowledge rather than deep diving into each piece, since for tasks like these, I limit my time to approx **3 hours**.

# Solution Overview

My philiosphy is to create abstract modular packages, import and use them in the notebook. If this was a 'real' project, there would be automated unit tests for each function/method.

The python module contains documented code describing in more details what is being done. Please check that out if you want to know what is being done in detail

# Note: I noticed Jupyterlab doesn't have a spell checker build in, apologies in advance

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from optum_challenge.preprocessing import DataReader, LabelReader

# This notebook contains the following and is 1 of 3

* Read in data and set schema, cleaning up some anomalies in the data.
* Initial data QA: removing fields that won't be going for visualisation due to time constraints
* Finding duplicated and modified data
* Removing some fields that are very similar to each other/subsets of each other
* Cleaning labels data and joining it to features data
* Writing file out for visualisation and modelling etc


# Reading data in correct format

I'm not a big fan of inferring schema from a csv file, I'd generally use something like parquet that can hold field metatdata. For that reason, I created a class in the optum_challenge module for setting schema for a safe data read. Some small modifications are done to the data, please check the DataReader class for more details

In [2]:
reader = DataReader('data.csv')

In [3]:
df = reader.clean_read()

In [4]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000,2011-03-14,Roman,1390.0,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,TRUE,GeoData Consultants Ltd,VWC,Roman,FALSE,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0,2013-03-06,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,TRUE,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25,2013-02-25,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,TRUE,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,TRUE,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0,2013-01-28,Unicef,263.0,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,TRUE,GeoData Consultants Ltd,VWC,NaN,TRUE,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0,2011-07-13,Action In A,0.0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,TRUE,GeoData Consultants Ltd,NaN,NaN,TRUE,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


# Data QA and finding low information/duplicate columns

* There are a large amount of features, I don't have time to visualise them all, so will try and reduce the number based on zero variance and duplicated (or near duplicate fields) or fields such as it's name that don't hold too much informatin

In [5]:
df.wpt_name.nunique()

37400

In [6]:
df.wpt_name.value_counts()

none                  3563
Shuleni               1748
Zahanati               830
Msikitini              535
Kanisani               323
                      ... 
Baba Sesi                1
Kwa Mzee Kimbawala       1
Kanisa La Lutheran       1
Kwa Isaak                1
Dip Makundi              1
Name: wpt_name, Length: 37400, dtype: int64

It is expected that each id is has only 1 entry

In [7]:
assert df.id.value_counts().max() == 1

AssertionError: 

There is a case where an id appears twice, let's investigate

In [8]:
df[df.id == '70713']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
275,70713,0,2013-02-14,Danida,458.0,DANIDA,34.735798,-1.116676e+01,Kwa Mzee Maluka,0,Lake Nyasa,Kawawa,Ruvuma,10,3,Mbinga,Lipingo,86,NaN,GeoData Consultants Ltd,VWC,DANIDA,TRUE,1990,gravity,gravity,gravity,vwc,user-group,unknown,unknown,soft,good,dry,dry,spring,spring,groundwater,communal standpipe,communal standpipe
44370,70713,0,2011-07-27,Rwssp,0.0,DWE,0.000000,-2.000000e-08,Tupendane,0,Lake Victoria,Nding'Ho B,Shinyanga,17,1,Bariadi,Mwaswale,0,TRUE,GeoData Consultants Ltd,WUG,NaN,FALSE,0,nira/tanira,nira/tanira,handpump,wug,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


Without having domain experience or deep knowledge of the data (hard to do that in 3 hours with the number of tasks in this challenge) these both look like valid points and it looks like a mistake was made by whoever collected the data. But in terms of validity, they both look fine. However, if when target data is joined, the same value could join to both records. This means, for training purposes, they may have to be removed but could be predicted on.

### Checking for any zero variance 

In [9]:
unique_col_counts = df.nunique()
unique_col_counts[unique_col_counts == 1]

recorded_by    1
dtype: int64

The field `recorded_by` only has 1 value and therefore won't be able help when modelling

### Picking one of similar fields

* There appears to be several fields that are extremely similar. To reduce the dimensionality and save time, I'll just pick one of them

#### Examining extraction type fields

In [10]:
df.extraction_type.value_counts(dropna=False, normalize=True)

gravity                      0.450851
nira/tanira                  0.137270
other                        0.108247
submersible                  0.080201
swn 80                       0.061783
mono                         0.048232
india mark ii                0.040403
afridev                      0.029797
ksb                          0.023821
other - rope pump            0.007592
other - swn 81               0.003855
windmill                     0.001970
india mark iii               0.001650
cemo                         0.001515
other - play pump            0.001431
walimi                       0.000808
climax                       0.000539
other - mkulima/shinyanga    0.000034
Name: extraction_type, dtype: float64

In [11]:
df.extraction_type_group.value_counts(dropna=False, normalize=True)

gravity            0.450851
nira/tanira        0.137270
other              0.108247
submersible        0.104022
swn 80             0.061783
mono               0.048232
india mark ii      0.040403
afridev            0.029797
rope pump          0.007592
other handpump     0.006128
other motorpump    0.002054
wind-powered       0.001970
india mark iii     0.001650
Name: extraction_type_group, dtype: float64

In [12]:
df.extraction_type_class.value_counts(dropna=False, normalize=True)

gravity            0.450834
handpump           0.277032
other              0.108247
submersible        0.104022
motorpump          0.050285
rope pump          0.007592
wind-powered       0.001970
infinity stones    0.000017
Name: extraction_type_class, dtype: float64

In [13]:
df.groupby('extraction_type_class')['extraction_type_group'].nunique().sort_values(ascending=False)

extraction_type_class
handpump           6
motorpump          2
wind-powered       1
submersible        1
rope pump          1
other              1
infinity stones    1
gravity            1
Name: extraction_type_group, dtype: int64

In [14]:
df.groupby('extraction_type_class')['extraction_type'].nunique().sort_values(ascending=False)

extraction_type_class
handpump           9
motorpump          3
submersible        2
wind-powered       1
rope pump          1
other              1
infinity stones    1
gravity            1
Name: extraction_type, dtype: int64

From examining the dataframe and table above, yes they are very similar so I'll pick the one with most levels, since the others are aggregates of it. If this was a real project, you could try each one in a model to figure out which is best, but here I just choose one.

Generally when modelling, if there are many levels to a feature, I'll cut instances that occupy, let's say <1% of the training data and transform them to something else like `other`. I notice there already is an other there so I may include them in that or create a new level. Not sure yet. A different type of encoding could also be done, instead of one hot encoding, binary encoding could be done. Or better still, use a method that doesn't require encoding and supports categorical features natively, i.e. LightGBM or XGBoost.

I also don't think `infinity stones` is an extraction type. If there are other nuggets in this dataset like this, I may not find them all as I'm not spending hours and hours going through each field.

#### Management

In [15]:
df.management.nunique()

12

In [16]:
df.management_group.nunique()

5

In [17]:
df.management.value_counts(dropna=False)

vwc                 40508
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [18]:
df.management_group.value_counts(dropna=False)

user-group    52491
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64

Again, without more domain knowledge, hard to say if these are describing the exact same thing. For that reason, I'll keep them in for now

#### Payment

In [19]:
df.payment.nunique()

7

In [20]:
df.payment_type.nunique()

8

In [21]:
df.payment.value_counts(dropna=False)

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8158
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [22]:
df.payment_type.value_counts(dropna=False)

never pay     25347
per bucket     8985
monthly        8300
unknown        8158
on failure     3914
annually       3642
other          1054
bitcoin           1
Name: payment_type, dtype: int64

Dropping `payment_type` since it just a copy with bitcoin added

#### water quality

In [23]:
df.water_quality.nunique()

8

In [24]:
df.quality_group.nunique()

6

In [25]:
df.groupby('quality_group')['water_quality'].nunique().sort_values(ascending=False)

quality_group
salty       2
fluoride    2
unknown     1
milky       1
good        1
colored     1
Name: water_quality, dtype: int64

In [26]:
df.quality_group.value_counts(dropna=False, normalize=True)

good        0.855524
salty       0.087456
unknown     0.031582
milky       0.013535
colored     0.008249
fluoride    0.003653
Name: quality_group, dtype: float64

In [27]:
df.water_quality.value_counts(dropna=False, normalize=True)

soft                  0.855524
salty                 0.081749
unknown               0.031582
milky                 0.013535
coloured              0.008249
salty abandoned       0.005707
fluoride              0.003367
fluoride abandoned    0.000286
Name: water_quality, dtype: float64

Looks like water quality is just a very similar subset of quality_group so will just take quality group going forward

#### Quantity

In [28]:
quant_same = pd.Series(np.where(df.quantity == df.quantity_group, 1, 0))

In [29]:
quant_same.nunique()

1

Picking `quantity` since it and `quantity_group` are duplicates

#### Source

In [30]:
df.source.nunique()

10

In [31]:
df.source_class.nunique()

3

In [32]:
df.source_type.nunique()

8

In [33]:
df.source.value_counts(dropna=False)

spring                  17022
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [34]:
df.source_class.value_counts(dropna=False)

groundwater    45795
surface        13328
unknown          278
Name: source_class, dtype: int64

In [35]:
df.source_type.value_counts(dropna=False)

spring                  17022
shallow well            16823
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
guinness                    1
Name: source_type, dtype: int64

In [36]:
df.groupby('source_type')['source'].nunique()

source_type
borehole                2
dam                     1
guinness                1
other                   2
rainwater harvesting    1
river/lake              2
shallow well            1
spring                  1
Name: source, dtype: int64

Dropping `source_type` since it subset and is similar to `source` but with added guiness

#### water point

In [37]:
df.waterpoint_type.nunique()

7

In [38]:
df.waterpoint_type_group.nunique()

6

In [39]:
df.waterpoint_type.value_counts(dropna=False)

communal standpipe             28523
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64

In [40]:
df.waterpoint_type_group.value_counts(dropna=False)

communal standpipe    34626
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

In [41]:
df.groupby('waterpoint_type_group')['waterpoint_type'].nunique()

waterpoint_type_group
cattle trough         1
communal standpipe    2
dam                   1
hand pump             1
improved spring       1
other                 1
Name: waterpoint_type, dtype: int64

Dropping `waterpoint_type_group` for similar reasons

#### Checking location features

In [42]:
df.basin.value_counts(dropna=False, normalize=True)

Lake Victoria              0.172506
Pangani                    0.150503
Rufiji                     0.134274
Internal                   0.131058
Lake Tanganyika            0.108281
Wami / Ruvu                0.100790
Lake Nyasa                 0.085621
Ruvuma / Southern Coast    0.075638
Lake Rukwa                 0.041312
Lake Victoriia             0.000017
Name: basin, dtype: float64

Spelling mistake added.

In [43]:
df.subvillage.value_counts(dropna=False, normalize=False).head(20)

Madukani      508
Shuleni       506
Majengo       502
Kati          373
NaN           371
Mtakuja       262
Sokoni        232
M             187
Muungano      172
Mbuyuni       164
Mlimani       152
Songambele    147
Miembeni      134
Msikitini     134
1             132
Kibaoni       114
Kanisani      111
I             109
Mapinduzi     109
Mjini         108
Name: subvillage, dtype: int64

There are would be many levels to `subvilliage`, meaning is too granular, one level up could be more useful and generalise better (and for time keeping reasons), I'm dropping this. 

In [44]:
df.region.value_counts(dropna=False, normalize=True).head(30)

Iringa           0.089123
Shinyanga        0.083871
Mbeya            0.078096
Kilimanjaro      0.073719
Morogoro         0.067440
Arusha           0.056396
Kagera           0.055824
Mwanza           0.052221
Kigoma           0.047407
Ruvuma           0.044461
Pwani            0.044360
Tanga            0.042878
Dodoma           0.037053
Singida          0.035235
Mara             0.033148
Tabora           0.032979
Rukwa            0.030437
Mtwara           0.029124
Manyara          0.026649
Lindi            0.026026
Dar es Salaam    0.013552
Name: region, dtype: float64

In [45]:
df.groupby('region')['region_code'].nunique().sort_values(ascending=False)

region
Pwani            3
Lindi            3
Shinyanga        3
Mtwara           3
Tanga            2
Mwanza           2
Arusha           2
Rukwa            1
Ruvuma           1
Morogoro         1
Tabora           1
Mara             1
Manyara          1
Singida          1
Kilimanjaro      1
Kigoma           1
Kagera           1
Iringa           1
Dodoma           1
Dar es Salaam    1
Mbeya            1
Name: region_code, dtype: int64

For most regions, there is only one code, except for a few, so I'll drop `region_code` from analysis.

In [46]:
df.district_code.value_counts(dropna=False, normalize=True)

1     0.205434
2     0.188094
3     0.168330
4     0.151496
5     0.073332
6     0.068585
7     0.056279
8     0.017559
30    0.016751
33    0.014714
53    0.012542
43    0.008502
13    0.006582
23    0.004933
63    0.003283
62    0.001835
60    0.001061
0     0.000387
80    0.000202
67    0.000101
Name: district_code, dtype: float64

In [47]:
df.district_code.nunique()

20

In [48]:
df.ward.nunique()

2092

Again, `ward` is very granular, having over 2000 levels. For the purposes of this, I'll drop it.

In [49]:
df.lga.nunique()

125

In [50]:
df.lga.value_counts(normalize=True).head(30)

Njombe           0.042137
Arusha Rural     0.021077
Moshi Rural      0.021060
Bariadi          0.019814
Rungwe           0.018619
Kilosa           0.018417
Kasulu           0.017626
Mbozi            0.017407
Meru             0.016986
Bagamoyo         0.016784
Singida Rural    0.016751
Kilombero        0.016145
Same             0.014764
Kibondo          0.014714
Kyela            0.014461
Kahama           0.014074
Kigoma Rural     0.013872
Magu             0.013872
Maswa            0.013619
Karagwe          0.012980
Mbinga           0.012643
Iringa Rural     0.012256
Serengeti        0.012054
Lushoto          0.011683
Namtumbo         0.011683
Songea Rural     0.011666
Mpanda           0.011431
Mvomero          0.011296
Ngara            0.011262
Ulanga           0.011195
Name: lga, dtype: float64

In [51]:
ward_count_per_lga = df.groupby('lga')['ward'].nunique().sort_values(ascending=False)

In [52]:
ward_count_per_lga.quantile(np.linspace(0,1,11))

0.0     1.0
0.1     9.0
0.2    11.0
0.3    14.0
0.4    16.0
0.5    17.0
0.6    18.4
0.7    20.0
0.8    24.2
0.9    26.0
1.0    37.0
Name: ward, dtype: float64

In [53]:
df.ward.nunique()

2092

# Dropping fields from above analysis

In [54]:
drop_fields = [
    'wpt_name',
    'recorded_by',
    'extraction_type_class',
    'extraction_type_group',
    'payment_type',
    'water_quality',
    'quantity_group',
    'source_type',
    'waterpoint_type_group',
    'region_code',
    'subvillage',
    'ward',    
]

In [55]:
df = df.drop(drop_fields, axis=1)

In [56]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,num_private,basin,region,district_code,lga,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,management,management_group,payment,quality_group,quantity,source,source_class,waterpoint_type
0,69572,6000,2011-03-14,Roman,1390.0,Roman,34.938093,-9.856322,0,Lake Nyasa,Iringa,5,Ludewa,109,TRUE,VWC,Roman,FALSE,1999,gravity,vwc,user-group,pay annually,good,enough,spring,groundwater,communal standpipe
1,8776,0,2013-03-06,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,0,Lake Victoria,Mara,2,Serengeti,280,NaN,Other,NaN,TRUE,2010,gravity,wug,user-group,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe
2,34310,25,2013-02-25,Lottery Club,686.0,World vision,37.460664,-3.821329,0,Pangani,Manyara,4,Simanjiro,250,TRUE,VWC,Nyumba ya mungu pipe scheme,TRUE,2009,gravity,vwc,user-group,pay per bucket,good,enough,dam,surface,communal standpipe multiple
3,67743,0,2013-01-28,Unicef,263.0,UNICEF,38.486161,-11.155298,0,Ruvuma / Southern Coast,Mtwara,63,Nanyumbu,58,TRUE,VWC,NaN,TRUE,1986,submersible,vwc,user-group,never pay,good,dry,machine dbh,groundwater,communal standpipe multiple
4,19728,0,2011-07-13,Action In A,0.0,Artisan,31.130847,-1.825359,0,Lake Victoria,Kagera,1,Karagwe,0,TRUE,NaN,NaN,TRUE,0,gravity,other,other,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe


# Joining labels to data for visualisations

* see LabelReader class in preprocessing.py to see what was done to prepare the data

In [57]:
label_reader = LabelReader('labels.csv')

In [58]:
label_df = label_reader.clean_read()

There was an id (70713) that had 2 records, both are dropped because when joining label, we can't be sure which one it corresponds too

In [59]:
df = df.drop_duplicates(subset='id', keep=False)

df_size_before_join = df.shape[0]

df_comb = df.merge(label_df, on='id', how='inner')

df_size_after_join = df_comb.shape[0]

# There were 2 labels we didn't have data for and are lost when inner joined
assert df_size_before_join == df_size_after_join + 2

In [60]:
df_comb.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,num_private,basin,region,district_code,lga,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,management,management_group,payment,quality_group,quantity,source,source_class,waterpoint_type,status_group
0,69572,6000,2011-03-14,Roman,1390.0,Roman,34.938093,-9.856322,0,Lake Nyasa,Iringa,5,Ludewa,109,TRUE,VWC,Roman,FALSE,1999,gravity,vwc,user-group,pay annually,good,enough,spring,groundwater,communal standpipe,functional
1,8776,0,2013-03-06,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,0,Lake Victoria,Mara,2,Serengeti,280,NaN,Other,NaN,TRUE,2010,gravity,wug,user-group,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe,functional
2,34310,25,2013-02-25,Lottery Club,686.0,World vision,37.460664,-3.821329,0,Pangani,Manyara,4,Simanjiro,250,TRUE,VWC,Nyumba ya mungu pipe scheme,TRUE,2009,gravity,vwc,user-group,pay per bucket,good,enough,dam,surface,communal standpipe multiple,functional
3,67743,0,2013-01-28,Unicef,263.0,UNICEF,38.486161,-11.155298,0,Ruvuma / Southern Coast,Mtwara,63,Nanyumbu,58,TRUE,VWC,NaN,TRUE,1986,submersible,vwc,user-group,never pay,good,dry,machine dbh,groundwater,communal standpipe multiple,non functional
4,19728,0,2011-07-13,Action In A,0.0,Artisan,31.130847,-1.825359,0,Lake Victoria,Kagera,1,Karagwe,0,TRUE,NaN,NaN,TRUE,0,gravity,other,other,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe,functional


In [61]:
df_comb.status_group.value_counts(normalize=True, dropna=False)

functional                 0.543091
non functional             0.384245
functional needs repair    0.072664
Name: status_group, dtype: float64

# writing out file

In [67]:
# df_comb.to_parquet('data_combined_for_eda')